# Øving 3, TMA4320

**Veiledning**: Onsdagene 08.03 og 15.03 kl. 16:15-18:00 i R50, Realfagsbygget, og torsdagene 09.03 og 16.03 i 14:15-16:00 i rom 265 SB1.  
**Innlevering**: Mandag 20.03 kl. 23:59, i [ovsys](https://ovsys.math.ntnu.no).

Oppgaven skal innleveres som et Jupyternotat. Men gjør gjerne implementering og koding i Spyder eller et annet IDE, og kopier den ferdige koden inn i Jupyternotatet for innlevering.

**NB!** Før innlevering: 
* Kjør en runde på hele notatet for å se at alt virker: <tt>Kernel -> Restart & Run All</tt>
* Deretter: <tt>Kernel -> Restart & Clear Output</tt>. Fila er nå klar for innlevering. 

$\newcommand{mb}[1]{\mathbf{#1}}$
$\newcommand{R}{\mathbb{R}}$
    

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
newparams = {'figure.figsize': (8.0, 4.0), 'axes.grid': True,
             'lines.markersize': 8, 'lines.linewidth': 2,
             'font.size': 14}
plt.rcParams.update(newparams)

### Oppgave 1
Gitt differensialligningen
$$
y' = ty^2, \qquad y(0)=1. 
$$
skal løses over intervallet $0\leq t \leq 0.4$.

**(a)** Finn den eksakte løsningen av ligningen

**(b)** Gjør 4 steg med Eulers metode med steglengde $h=0.1$. Beregn feilen i endepunktet.

**(c)** Gjør 2 steg med Heun's metode med steglengde $h=0.2$. Beregn igjen feilen i endepunktet. 

**(d)** Gjør et steg med RK4. Beregn feilen i endepunktet. 

I alle disse tilfellene er det brukt 4 funksjonsevalueringer for å finne en tilnærmelse til løsningen $y(0.4)$. Hvem av metodene ga et mest nøyaktig resultat? 


Finner analytisk ved å bruke WolframAlpha $$ y = -\frac{2}{C + t^2} $$

In [ ]:
def f(t, y):
    return t*y**2


def y(t, y0):
    return -2/(-2/y0 + t**2)


def euler(f, t0, y0, tend, N=100):
    '''
    Euler's method for solving y'=f(t,y), y(t0)=y0, as given in the lecture notes.
    '''
    h = (tend-t0)/N         # Stepsize

     
    # In the case of a scalar ODE, convert y0 to a numpy vector.
    if not isinstance(y0, np.ndarray): 
        y0 = np.array([y0])
        m = 1
    else:
        m = len(y0)
    
    # Make arrays for storing the solution. 
    ysol = np.zeros((N+1, m))
    tsol = np.zeros(N+1)
    # Insert the initial values
    ysol[0,:] = y0
    tsol[0] = t0

    tn = t0
    yn = y0

    # Main loop
    for n in range(N):
        # One step of Euler's method
        yn = yn+h*f(tn,yn)
        tn = tn+h

        # Store the solution
        ysol[n+1,:] = yn
        tsol[n+1] = tn

    # In case of a scalar problem, convert the solution into a 1d-array
    if m==1:
        ysol = ysol[:,0] 

    return tsol, ysol

In [ ]:
def heun_euler(f, t0, y0, tend, N = 100):
    '''
    Heun's method for solving y'=f(t,y), y(t0)=y0.
    '''
    h = (tend-t0)/N
    
    if not isinstance(y0, np.ndarray): 
        y0 = np.array([y0])
        m = 1
    else:
        m = len(y0)
    
    ysol = np.zeros((N+1, m))
    tsol = np.zeros(N+1)
        
    ysol[0,:] = y0
    tsol[0] = t0
    
    yn = y0
    tn = t0
    
    for i in range(N):
        k1 = f(tn, yn)
        k2 = f(tn + h, yn + h*k1)
        yn = yn + h/2 * (k1 + k2)
        tn = tn + h
        
        ysol[i+1,:] = yn
        tsol[i+1] = tn
    
    if m==1:
        ysol = ysol[:,0]
        

    return tsol, ysol

In [ ]:
def RK4(f, t0, y0, tend, N):
    """
    RK4 method for solving y'=f(t,y), t(t0)=y0.
    """
    h = (tend-t0)/N
    
    if not isinstance(y0, np.ndarray): 
        y0 = np.array([y0])
        m = 1
    else:
        m = len(y0)
    
    ysol = np.zeros((N+1, m))
    tsol = np.zeros(N+1)
    
    ysol[0,:] = y0
    tsol[0] = t0
    
    yn = y0
    tn = t0
    
    for i in range(N):
        k1 = f(tn, yn)
        k2 = f(tn + h/2, yn + h*k1/2)
        k3 = f(tn + h/2, yn + h*k2/2)
        k4 = f(tn + h, yn + h*k3)
        
        yn = yn + h * (k1/6 + k2/3 + k3/3 + k4/6)
        tn = tn + h
        
        ysol[i+1,:] = yn
        tsol[i+1] = tn
    
    if m==1:
        ysol = ysol[:,0]
    
    return tsol, ysol

In [ ]:
t0, y0, tend = 0, 1, 0.4
N = 100
t = np.linspace(t0, tend, N)
y_anal = y(t, y0)

N = 4
t_euler, y_euler = euler(f, t0, y0, tend, N)

N = 2
t_heun, y_heun = heun_euler(f, t0, y0, tend, N)

N = 1
t_RK, y_RK = RK4(f, t0, y0, tend, N)


plt.plot(t, y_anal, label="Analytical solution")
plt.plot(t_euler, y_euler, label="Euler's method")
plt.plot(t_heun, y_heun, label="Heun's method")
plt.plot(t_RK, y_RK, label="RK4 method")
plt.title("Numerical solutions to the differential equation $y' = ty^2$")
plt.xlabel('$t$')
plt.ylabel('$y$')
plt.legend()
plt.show()

print(f"""
The calculated errors using the analytic solution:
Euler's method: {abs(y_euler[-1]-y_anal[-1])}plt.plot(t_heuler, y_heuler[:,0])
plt.plot(y_heuler[:,1], t_heuler)
Heun's method: {abs(y_heun[-1]-y_anal[-1])}
RK4 method: {abs(y_RK[-1]-y_anal[-1])}
""")

### Oppgave 2

Følgende Runge-Kutta metode er gitt ved sitt Butcher-tableau:
$$
\begin{array}{c|ccc}
  0 & 0 \\
  1/3 & 1/3 \\
  2/3 & 0 & 2/3 \\ \hline
      & 1/4 & 0 & 3/4 
\end{array}
$$

**(a)** Bestem metodens orden. 

**(b)** Verifiser metodens orden numerisk. Du kan bruke testligningen fra Oppgave 1. 

Metodens er av 3. orden.

In [ ]:
def RK3(f, t0, y0, tend, N):
    h = (tend-t0)/N

    ysol = np.zeros(N+1)
    tsol = np.zeros(N+1)
    
    ysol[0] = y0
    tsol[0] = t0
    
    yn = y0
    tn = t0
    
    for i in range(N):
        k1 = f(tn, yn)
        k2 = f(tn + h/3, yn + h*k1/3)
        k3 = f(tn + 2*h/3, yn + 2*h*k2/3)

        yn += h * (k1/4 + 3*k3/4)
        tn += h
        
        ysol[i+1] = yn
        tsol[i+1] = tn

    return tsol, ysol


t0, y0, tend = 0, 1, 0.4
N = 2
t_RK, y_RK = RK3(f, t0, y0, tend, N)
plt.plot(t, y_anal, label="Analytical solution")
plt.plot(t_RK, y_RK, label="Runge-Kutta method")
plt.legend()
plt.xlabel('$t$')
plt.ylabel('$y$')
plt.title("Numerical solution using given Butcher-tableau for $y' = ty^2$")
plt.show()

### Oppgave 3
Duffing-ligningen er en mye studert matematisk modell, som beskriver en tynn, bøyelig metallstav som svinger i nærheten av en elektromagnet. Ligningen er gitt ved

$$
u''+ku' - u(1-u^2)=A\cos(\omega t).
$$

**(a)** Skriv om ligningen til et system av differensialligninger. 

I det resterende kan du bruke parameterne $k=0.25$, $A=0.4$, $\omega = 1.0$, $u(0)=0$, $u'(0)=0$

**(b)** Gjør et steg med Heuns metode. Bruk steglengde $h=0.1$.

**(c)** Løs ligningen numerisk ved hjelp av Heuns metode. Du kan ta utgangspunkt i koden <tt>ode.py</tt>, som du kan finne på forelesningsplanen. Bruk $h=0.01$, og integrer fram til $t_{\text{end}}=100$. Plott begge løsningskomponentene som en funksjon av $t$. Vis deretter løsningen som et faseplott ($u$ på $x$-aksen, $u'$ på $y$-aksen). 

Eksprimenter gjerne med litt ulike startverdier og parameterverdier, men bruk verdiene som er oppgitt i innleveringen.

**(d)** Løs ligningen over samme intervall som du brukte i **(c)**, men nå med <tt> scipy</tt>'s <tt>solve_ivp</tt>. Løs ligningene med litt ulike toleranser (<tt>atol</tt> og <tt>rtol</tt>), og se hva som skjer. 

Duffing-oscillatoren er et eksempel på et kaotisk system. Små endringer kan føre til ganske store endringer i løsningen, men løsningene holder seg likevel innenfor nogenlunde samme område. 

Skriver om til likningssystemet
\begin{equation}
\begin{split}
    y_1' &= A\cos \omega t + y_2 (1 - y_2^2) - ky_1 \\
    y_2' &= y_1 \\
\end{split}
\end{equation}

In [ ]:
k, A, w = 0.25, 0.4, 1.0
y0 = np.array([0, 0])

def g(t, y):
    y1 = A*np.cos(w*t) + y[1]*(1 - y[1]**2) - k*y[0]
    y2 = y[0]
    return np.array([y1, y2])


t0, tend = 0, 0.1
N = 1
t_heun, y_heun = heun_euler(g, t0, y0, tend, N)

plt.plot(t_heun, y_heun, label="Heun's method")
plt.title("Heun's method to solve given differential equations")
plt.xlabel('$t$')
plt.ylabel('$y$')
plt.show()

print(t_heun, y_heun)

In [ ]:
def heun_euler_lecture(f, t0, y0, tend, h0, tol=1.e-6):
    '''
    Heun-Eulers's adaptive method for solving y'=f(t,y), y(t0)=y0, as given in the lecture notes.
    '''
     
    # In the case of a scalar problem, convert y0 to a numpy vector.
    if not isinstance(y0, np.ndarray): 
        y0 = np.array([y0])
        m = 1
    else:
        m = len(y0)
   
    ysol = np.array([y0]) # Arrays to store the solution
    tsol = np.array([t0])

    yn = y0
    tn = t0
    h = h0
    MaxNumberOfSteps = 100000  # Maximum number of steps, accepted and rejeced
    ncount = 0

    # Main loop
    while tn < tend - 1.e-10:
        if tn+h > tend:
            h = tend - tn
        
        # One step with Heun's method
        k1 = f(tn,yn)
        k2 = f(tn+h, yn+h*k1)

        # Calculate the error estimate
        le_n = 0.5*h*np.linalg.norm(k2-k1)   # Measured in the 2-norm
        
        
        if le_n <= tol:             
            # Solution accepted, update tn and yn
            yn = yn+0.5*h*(k1+k2)
            tn = tn+h
            # Store the solution
            ysol = np.concatenate((ysol, np.array([yn])))
            tsol = np.append(tsol, tn)
        # else the step is rejected and nothing is updated. 

        # Change the stepsize
        h = 0.8*(tol/le_n)**(1/2)*h
        
        ncount += 1
        if ncount > MaxNumberOfSteps:
            raise Exception('Maximum number of steps reached')
  
    # In case of a scalar problem, convert the solution into a 1d-array
    if m==1:
        ysol = ysol[:,0] 

    return tsol, ysol

h0 = 0.01
t0, tend = 0, 100
y0 = np.array([0, 0])

t_heuler, y_heuler = heun_euler_lecture(g, t0, y0, tend, h0, tol=1.e-4)

plt.plot(t_heuler, y_heuler[:,0])
plt.title("Solution to given differential equations using handed out Heun-Euler's method")
plt.xlabel('$t$')
plt.ylabel('$y$')
plt.show()

In [ ]:
sol = solve_ivp(g, (t0, tend), y0)

plt.plot(sol.t, sol.y[0])
plt.title("Solution to given differential equations using scipy.integrate.solve_ivp()")
plt.xlabel('$t$')
plt.ylabel('$y$')
plt.show()

In [ ]:
def phase_portrait(g, xmin, xmax, ymin, ymax, n):
    """
    Plots a phase portrait, code found at https://kitchingroup.cheme.cmu.edu/blog/2013/02/21/Phase-portraits-of-a-system-of-ODEs/ .
    """
    y1 = np.linspace(xmin, xmax, n)
    y2 = np.linspace(ymin, ymax, n)
    
    Y1, Y2 = np.meshgrid(y1, y2)
    
    t = 0
    
    u,v = np.zeros(Y1.shape), np.zeros(Y2.shape)
    
    NI, NJ = Y1.shape
    
    for i in range(NI):
        for j in range(NJ):
            x = Y1[i, j]
            y = Y2[i, j]
            z1, z2 = g(t, [x, y])
            u[i, j] = z1
            v[i, j] = z2
    Q = plt.quiver(Y1, Y2, u, v)
    
    plt.xlabel('$u$')
    plt.ylabel("$u'$")
    plt.title("Phase portrait for solutions to given differential equations")
    
    plt.show()

phase_portrait(g, -5, 5, -2, 2, 15)